In [5]:
from pymongo import MongoClient
from config import username, password, host, database_name
import json
from urllib.parse import quote_plus
import os

def connect_to_mongodb():
    """
    Connects to the MongoDB instance.
    """
    try:
        # Escape the username and password
        escaped_username = quote_plus(username)
        escaped_password = quote_plus(password)

        # Create the connection string using an F-string
        uri = f"mongodb+srv://{escaped_username}:{escaped_password}@{host}/?retryWrites=true&w=majority"

        # Connect to the MongoDB instance
        client = MongoClient(uri)

        print("Connected to MongoDB")
        return client

    except Exception as e:
        print(f"Error connecting to MongoDB: {e}")
        return None

def create_collection(database, collection_name):
    """
    Creates a MongoDB collection.
    """
    try:
        # Access the specified database
        db = database[database_name]

        # Create the collection if it doesn't exist
        collection = db[collection_name]
        print(f"Collection '{collection_name}' created.")
        return collection

    except Exception as e:
        print(f"Error creating collection: {e}")
        return None

def find_geojson_in_same_folder():
    """
    Finds a GeoJSON file in the same folder as the script.
    """
    script_dir = os.path.dirname(os.path.abspath(__file__)) if "__file__" in locals() else os.getcwd()
    geojson_files = [file for file in os.listdir(script_dir) if file.endswith('.geojson')]

    if geojson_files:
        # If multiple files found, prompt the user to choose one
        if len(geojson_files) > 1:
            print("Multiple GeoJSON files found in the same folder:")
            for i, file in enumerate(geojson_files, start=1):
                print(f"{i}. {file}")

            choice = int(input("Enter the number of the file you want to use: "))
            chosen_file = geojson_files[choice - 1]
        else:
            chosen_file = geojson_files[0]

        file_path = os.path.join(script_dir, chosen_file)
        return file_path

    else:
        print("No GeoJSON file found in the same folder.")
        return None

def load_geojson_data(file_path):
    """
    Loads GeoJSON data from a file.
    """
    try:
        with open(file_path, 'r') as file:
            geojson_data = json.load(file)
        return geojson_data

    except Exception as e:
        print(f"Error loading GeoJSON data: {e}")
        return None

def insert_geojson_data(collection, geojson_data):
    """
    Inserts GeoJSON data into a MongoDB collection.
    """
    try:
        # Insert GeoJSON data into the collection
        collection.insert_many(geojson_data['features'])
        print(f"GeoJSON data inserted into MongoDB collection.")
        return True

    except Exception as e:
        print(f"Error inserting GeoJSON data: {e}")
        return False

# Automatically find a GeoJSON file in the same folder
file_path = find_geojson_in_same_folder()

if file_path:
    # Connect to MongoDB
    mongo_client = connect_to_mongodb()

    if mongo_client:
        # Specify the collection name
        collection_name = 'ZIP_Polygon_KS_Test'

        # Create the 'ZIP_Polygon_KS_Test' collection
        ZIP_Polygon_KS_Test = create_collection(mongo_client, collection_name)

        # Check if the collection was created successfully
        if ZIP_Polygon_KS_Test is not None:
            # Load GeoJSON data from the file
            geojson_data = load_geojson_data(file_path)

            if geojson_data:
                # Insert GeoJSON data into the 'ZIP_Polygon_KS_Test' collection
                insert_geojson_data(ZIP_Polygon_KS_Test, geojson_data)

    # Close the MongoDB connection
    if mongo_client:
        mongo_client.close()
        print("Disconnected from MongoDB")


Connected to MongoDB
Collection 'ZIP_Polygon_KS_Test' created.
GeoJSON data inserted into MongoDB collection.
Disconnected from MongoDB
